<a href="https://colab.research.google.com/github/vanderbilt-data-science/ai_summer/blob/main/4_2_ift_rlhf_wb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning (SFT) and Reinforcement Learning from Human Feedback (RLHF)
> What happens after pre-training?

**Lesson objectives:** In this lesson, you will:
* Gain a high-level understanding of the total (known) steps of training large language models (LLMs) such as GPT3, Bard, etc.
* Gain conceptual understanding of SFT, Instruction Fine-Tuning (IFT), and RLHF
* Understand the requirements of SFT, IFT, and RLHF, including:
  * Data
  * Human effort
  * Code
  * Compute
* Implement components of SFT, IFT, and RLHF using open-source models on HF
* Gain a high-level view of optimizations that can help with training LLMs

Please feel free to contact me (Charreau Bell) or any VU DSI Team members for questions or updates/changes to the notebook!



In [ ]:
! pip install -q transformers datasets trl xformers

# Training of LLMs: Conceptual

## Recalling past conversations
[What makes a dialog agent useful?](https://huggingface.co/blog/dialog-agents)

Well, what does it take to make a dialog agent useful?
* 
*  
*  
*  

## Steps of LLM Training

0. Pre-training
1. Supervised Fine-Tuning (SFT)  
  a) Instruction Fine-Tuning (IFT)  
  b) Supervised Fine-Tuning (Safety)  
2. Training Reward/Preference Model
3. Reinforcement Learning from Human Feedback (RLHF)

<center>
<img src="https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/blog/133_trl_peft/openai-diagram.png" alt="Girl in a jacket" width="800">
</center>

<a href="https://arxiv.org/pdf/2203.02155.pdf">Source: Training language models to follow instructions with human feedback, OpenAI</a>


### Pre-training
<center>
<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg" alt="Pre-training" width="800">
</center>

<p>Source: <a href="https://huggingface.co/learn/nlp-course/chapter2/2?fw=pt#model-heads-making-sense-out-of-numbers">Huggingface NLP Course, Chapter 2.2: Behind the Pipeline</a></p>

Also see: [Improving Language Understanding
by Generative Pre-Training, OpenAI](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf)

***


### Supervised and Instruction Fine-Tuning
* [Self-instruct (Alpaca)](https://arxiv.org/abs/2212.10560)  
* [Unnatural Instructions](https://huggingface.co/datasets/mrm8488/unnatural-instructions)

***


### RLHF
<center>
<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/rlhf/rlhf.png" alt="RLHF" width="600">
</center>

* [Huggingface: Illustrated RLHF](https://huggingface.co/blog/rlhf)

***


## Putting it all together: StackLlama
[A demo from Huggingface](https://huggingface.co/blog/stackllama)
***


# Training of LLMs: Code

## Pre-training
We can use an existing pre-trained model (off-the-shelf) to start with, or pretrain our model some more.

**Data**:
* Your own
* Huggingface Datasets
* [RedPajama, open source LLama dataset](https://huggingface.co/datasets/togethercomputer/RedPajama-Data-1T)

**Model**:
* Your own or some custom model architecture (may not be friendly with HF API)
* Huggingface Models

**Tooling**:
* Framework (Huggingface, PyTorch, TF)

In [ ]:
#imports for loading data
from datasets import Dataset, load_dataset
import os

#imports for training
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer

#imports for evaluation(ish)
from transformers import pipeline

In [ ]:
#create/download pre-trained models
gpt2_tokenizer = AutoTokenizer.from_pretrained('gpt2')
gpt2 = AutoModelForCausalLM.from_pretrained('gpt2')

#avoid annoying errors
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
gpt2.config.pad_token_id = gpt2.config.eos_token_id

### Load custom data

In [ ]:
# download the data using command line tools
!curl -Os http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz
!tar -zxf 20news-18828.tar.gz

In [ ]:
# make dataset of interest
base_path = '20news-18828/sci.electronics/'
data_files = os.listdir(base_path)
text = []

# read all texts
for textfile in data_files:
  with open(''.join([base_path, textfile]), 'r', encoding='cp1252') as f: 
    text.append(f.read())

# make dataset


# split data


In [ ]:
# check


### Prepare for training

In [ ]:
# tokenize
def prepare_inputs(example):
  return gpt2_tokenizer(example["text"], truncation=True)

token_ds = hf_ds.map(prepare_inputs, batched=True, remove_columns='files')

Map:   0%|          | 0/784 [00:00<?, ? examples/s]

Map:   0%|          | 0/197 [00:00<?, ? examples/s]

In [ ]:
# make collator
data_collator = DataCollatorForLanguageModeling()

# training arguments
training_args = TrainingArguments(output_dir = "", #where the model should be saved
                                  logging_strategy = "",
                                  evaluation_strategy="", #how often to evaluate performance
                                  save_strategy='',
                                  logging_steps = ,
                                  eval_steps = , 
                                  save_steps = ,
                                  load_best_model_at_end = True,
                                  per_device_train_batch_size = ,
                                  per_device_eval_batch_size = ,
                                  num_train_epochs=,
                                  push_to_hub=, #whether or not to push to hub
                                  hub_strategy='checkpoint',
                                  report_to='all')

### Train

In [ ]:
#set data and functionality for trainer
gpt2 = AutoModelForCausalLM.from_pretrained('gpt2')
gpt2.config.pad_token_id = gpt2.config.eos_token_id

trainer = Trainer(model=,
                  args=,
                  tokenizer=,
                  data_collator=,
                  train_dataset=,
                  eval_dataset=
                  )

trainer.train()

## Supervised Fine-Tuning
What tools and materials do we need for SFT and IFT?

**Data**:
* Some instructions and/or safety fine-tuning data, or other fine-tuning data*
* [Instruction fine-tuning datasets (Alpaca)](https://huggingface.co/datasets/tatsu-lab/alpaca)

**Pre-trained model**

**Tooling**:
* Standard Huggingface transformer API, or 
*[Upcoming: HF SFT](https://huggingface.co/docs/trl/main/en/sft_trainer#trl.SFTTrainer)


In [ ]:
#load dataset and split for ease
#ift_ds = ?
ift_ds = ift_ds.train_test_split(train_size=0.8)

#tokenize
ift_token = ift_ds.map(prepare_inputs, batched=True, remove_columns=['instruction', 'input', 'output'])

In [ ]:
# training arguments
training_args = TrainingArguments(output_dir = "", #where the model should be saved
                                  logging_strategy = "steps",
                                  evaluation_strategy="steps", #how often to evaluate performance
                                  save_strategy='steps',
                                  logging_steps = ,
                                  eval_steps = , 
                                  save_steps = ,
                                  load_best_model_at_end = True,
                                  per_device_train_batch_size = ,
                                  per_device_eval_batch_size = ,
                                  num_train_epochs=1,
                                  push_to_hub=False, #whether or not to push to hub
                                  hub_strategy='checkpoint',
                                  report_to='all')

trainer = Trainer(model=gpt2,
                  args=training_args,
                  tokenizer=gpt2_tokenizer,
                  data_collator=data_collator,
                  train_dataset=ift_token['train'],
                  eval_dataset=ift_token['test']
                  )

trainer.train()

## Quick side quest: evaluating and comparing models
* Is the model better...practically?

In [ ]:
# Create test phrase
test_phrase = """I am writing a college paper on the history of world exploration.
Please write a 100-word introductory paragraph on this topic."""

In [ ]:
# Create pipelines to try for ease of use
init_gpt = pipeline('text-generation', 'gpt2', device='cuda:0')
ft_gpt = pipeline('text-generation',...)
sft_gpt = pipeline('text-generation',...)

#get responses
responses = [mdl(test_phrase, max_new_tokens=150) for mdl in [init_gpt, ft_gpt, sft_gpt]]


In [ ]:
for ind, model_name in enumerate(['Init gpt', 'Fine-tuned gpt', 'Instruction Fine-tuned gpt']):
  print(model_name, ':\n', responses[ind][0]['generated_text'], '\n\n*************\n')


## RLHF
What tools and materials do we need to have or need to create to implement RLHF?

**Data**:
* Prompts and answers with numeric reward associated with them

**Models**:
* To create or obtain: preference model (PM)/reward model(RM)
* SFT/IFT model

**Tooling**:
* [Huggingface TRL library](https://huggingface.co/docs/trl/index)

### Demo notebook for RLHF
* [Demo code](https://huggingface.co/docs/trl/sentiment_tuning)

***


# Congratulations!!
You should now have a high-level view of the world of training LLMs and some resources to help you do so. You should be able to state:
1. What are the common steps of training LLMs currently?
2. What happens during each of the common steps of training LLMs?
3. What tools are available to me to help me train my own model?
4. About how much data do I need for each step?
5. What human resources are needed to make this possible?

***



# Homework
You should read through:
* [Training language models to follow instructions with human feedback, OpenAI](https://arxiv.org/pdf/2203.02155.pdf), as well as their blog entry [here](https://openai.com/research/instruction-following). You should look for information including:
  - Exactly how much data was used/how many examples?
  - To what extent were human resources used in the training and how?
  - In addition to the _What Makes a Dialog Agent Useful_ blog above, how is safety implemented in OpenAI models? What about other models? What kinds of safety will you need for your application?
  - What types of RLHF preference models can I use?

Read through the other resources provided, specifically:
* The OpenAI pre-training paper
* The RHLF blog post
* Either the Alpaca (Instruction Fine-Tuning) paper linked above or [The Alpaca Blog post](https://crfm.stanford.edu/2023/03/13/alpaca.html)

Especially consider:
* Which steps do I or do I not need in this process for my application?
* How should I evaluate my model?
* What kind of data will I need?